<a href="https://colab.research.google.com/github/uttam112/sprinklr/blob/main/app_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation


1.   langchain
2.   Sentence Tranformers (Embeddings)
3.   faiss (Vector store)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install langchain

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss

#Loading Embeddings from Google Drive

In [ ]:
import pickle
with open("/content/drive/MyDrive/Embeddings/rpurified_only_Sprinklr1000_200.pkl", "rb") as f:
    db = pickle.load(f)

In [ ]:
!pip install openai

#Give your API Key

In [ ]:
import openai
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = "{Your_OpenAI_API_Key}"

#Conversation Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
from langchain.llms import OpenAI

In [ ]:
memory = ConversationBufferWindowMemory(k=3)

In [ ]:
llm = ChatOpenAI(temperature=0.0)
conversation = ConversationChain(
    llm=llm,
    memory = memory
)

print(conversation.prompt.template)
docs = ""
print('\n\n')
# conversation.prompt.template = str("You need to behave as Sprinklr sales agent. You are currently doing the role of Sprinklr AI Assistant. You need to engage the vistors and politely pursuade them to buy our product. \n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:")
print(conversation)
print(type(conversation))

#Template - Stand Alone Query

In [ ]:
conv = []
convo = ""
message = ""
chats_his = []
llm_primary = OpenAI(temperature=0.0)

In [ ]:
def args_temp():
  kil_template = f"""Given a snippet of previous dialogue of Sprinklr sales chatbot and an upcoming user question, convert the question into a stand-alone query. Don't change the upcoming question if it is not related to sprinklr:
  Previous conversation:
  <actual previous conversation with sprinklr bot here>
  Upcoming question:
  <actual upcoming user question here>
  Stand-alone query:
  <converted stand-alone question here>


  For example:[
  example1:
  Previous conversation:
  []
  Upcoming question:Hi
  Stand-alone query:Hi
  // previous conversation is blank so this the start of the conversation
  example2:
  Previous conversation:
  ['HumanMessage:Tell me about sprinklr modern marketing',
  Sprinklr Bot Message:Sprinklr's modern marketing services include a content management system that enables organizations to provide the correct material to their consumers at the right time, as well as a variety of other features such as Digital Asset Management (DAM), Workflow Automation, Content Analysis, and more.
  ]
  Upcoming question:tell me more
  Stand-alone query: What other features does Sprinklr's modern marketing services offer?


  example3:
  Previous conversation:
  []
  Upcoming question:what is 2+2?
  Stand-alone query:what is 2+2?
  // this query is not related to sprinklr so let it be unchanged.
  example4:
  Previous conversation:
  ['HumanMessage:Tell me about sprinklr modern marketing',
  Sprinklr Bot Message:Sprinklr's modern marketing services include a content management system that enables organizations to provide the correct material to their consumers at the right time, as well as a variety of other features such as Digital Asset Management (DAM), Workflow Automation, Content Analysis, and more.
  ]
  Upcoming question:Who is virat Kohli?
  Stand-alone query: Who is virat Kohli?
  // this query was not related to sprinklr so let it be unchanged.


  example5:
  Previous conversation:
  ['HumanMessage:tell me about sprinklr services',
  Sprinklr Bot Message:Sprinklr offers a range of services to help businesses manage their social media presence. These include social listening, social publishing, social advertising, social engagement, and social analytics. Our platform is designed to help businesses of all sizes and industries improve their social media strategy and drive better results. Would you like more information on any of these services?
  ]
  Upcoming question:what else?
  Stand-alone query: What are some other services that Sprinklr Provides?
  ]
  Previous conversation:
  [{convo}]
  Upcoming question:{message}
  Stand-alone query: """
  print(convo)
  return kil_template


#Gradio App

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
# message = "what are sprinklr services?"
# docs = db._similarity_search_with_relevance_scores(message,k =3)
# context = docs[0][0].page_content + "\n" + docs[1][0].page_content + "\n"+ docs[2][0].page_content
# relavance_scores = {"doc1":docs[0][1], "doc2":docs[1][1] , "doc3":docs[2][1]}
# template = "You need to behave as Sprinklr sales agent. You are currently doing the role of Sprinklr AI Assistant. You need to engage the visitors and politely pursuade them to buy our product.You must extract Sprinklr's data from the context (as the context may contain data of other orgainzations as well) to answer from the context under triple backtiks-```" + context + "```  \n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:"
# conversation.prompt.template = template
# bot_message = conversation.predict(input=message) + "\n\n" + context + "\n\n" + str(relavance_scores)

In [ ]:
# print(bot_message)

In [ ]:
# conv = []
# message = "hi"
# llm_primary = OpenAI(temperature=0.0)

In [ ]:
# message = "what do you provide?"
# convo = ""
# kil_template = args_temp()
# stand_alone_query = llm_primary.predict(kil_template)

In [ ]:
# print(stand_alone_query)

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    def respond(message, chat_history):



        convo = str(conv)
        kil_template = args_temp()
        stand_alone_query = llm_primary.predict(kil_template)
        """
        Will make a stand-alone query
        with the given upcoming question
        and the provided previous conversation
        """

        docs = db._similarity_search_with_relevance_scores(stand_alone_query,k =3)
        context = docs[0][0].page_content + "\n" + docs[1][0].page_content + "\n"+ docs[2][0].page_content
        """
        Concatenates top 3 most relevant documents
        to provide it as context for the bot to answer
        """






        relavance_scores = {"doc1":docs[0][1], "doc2":docs[1][1] , "doc3":docs[2][1]}

        template = "You need to behave as Sprinklr sales agent. You are currently doing the role of Sprinklr AI Assistant.  You should answer by referring context under triple backtiks-```" + context + "``` . If the question is not about Sprinklr, you must politely inform them that you are tuned to only answer questions about Sprinklr. \n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:"
        conversation.prompt.template = template
        bot_message = conversation.predict(input=message)



        if(len(conv) == 6):
           conv.pop(0)
           conv.pop(0)
        """
        Populates the conversation history
        with the current conversation
        Had a window size of 3
        """
        conv.append(f"HumanMessage:{message}\n")
        conv.append(f"Sprinklr Bot Message:{bot_message}\n")

        chat_history.append((message, bot_message))
        return "", chat_history
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True,debug=True)
